In [7]:
from util import read_csv

In [8]:
def is_prime(n):
    if n < 2:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

In [9]:
df = read_csv('data', 'combined_data_set.csv')

x:\Documents\Programming\dawr\dawr-grisons\src\util.py:45: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df.replace('---', pd.np.nan, inplace=True)


In [10]:
prime_numbers = df['Famous_Person_count'].apply(is_prime)

df_prime = df[prime_numbers]
df_prime.head()

,Gemeinde_Name,Einwohner,Einwohneraenderung,Bevoelkerungsdichte_pro_km2,Auslaender_in_prozent,Zero_to_Nineteen,Twenty_to_Sixtyfour,Sixtyfive_and_more,Marriage,Divorce,...,entity_id,counted,yeas,nays,invalid,empty,eligible_voters,expats,Famous_Person_list,Famous_Person_count
1,Lantsch/Lenz,539,1.315789,24.713434,17.439703,16.697588,60.111317,23.191095,1.862197,1.862197,...,3513,True,171,101,3,2,394,0.0,"Hans Ardüser (1557 – nach 1614), Bündner Wande...",3.0
17,Scharans,790,-2.829028,55.283415,3.417722,21.012658,56.835443,22.151899,1.266624,1.266624,...,3638,True,276,194,3,3,657,0.0,Bernardo da Poschiavo (* um 1475 in Scharans ?...,5.0
18,Sils im Domleschg,960,9.714286,103.448276,16.770833,23.229167,54.791667,21.979167,4.203889,0.000000,...,3640,True,258,193,0,4,656,0.0,Thomas Conrad von Baldenstein (1784–1878) auf ...,2.0
19,Cazis,2264,12.749004,72.610648,16.519435,19.920495,59.849823,20.229682,3.116652,2.226180,...,3661,True,541,504,3,2,1658,0.0,"Johann Anton Rizzi (1825–1868), Porträtmaler u...",2.0
21,Masein,498,21.760391,118.571429,5.220884,22.891566,60.843373,16.265060,0.000000,0.000000,...,3663,True,171,103,2,1,394,0.0,"Petrus Domenicus Rosius à Porta (1734–1806), r...",2.0


In [11]:
famous_count_surses = df[df['Gemeinde_Name'] == 'Surses']['Famous_Person_count'].reset_index(drop=True)
famous_count_vaz = df[df['Gemeinde_Name'] == 'Vaz/Obervaz']['Famous_Person_count'].reset_index(drop=True)

print(famous_count_surses)
print(famous_count_vaz)

print (famous_count_vaz > famous_count_surses)


0    0.0
Name: Famous_Person_count, dtype: float64
0    10.0
Name: Famous_Person_count, dtype: float64
0    True
Name: Famous_Person_count, dtype: bool


In [ ]:
# Ranking stuff
# normalize the factors using min-max normalization
normalized = (df[['GPA', 'Test Scores', 'Attendance', 'Participation']] 
              - df[['GPA', 'Test Scores', 'Attendance', 'Participation']].min()) / (df[['GPA', 'Test Scores', 'Attendance', 'Participation']].max() 
                   - df[['GPA', 'Test Scores', 'Attendance', 'Participation']].min())

# combine the normalized factors into a single score
df['Ranking Score'] = normalized.mean(axis=1) * 100